# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119246e+02     1.583163e+00
 * time: 0.12472319602966309
     1     1.064914e+01     1.167712e+00
 * time: 0.9729371070861816
     2    -1.193218e+01     1.201780e+00
 * time: 1.0969090461730957
     3    -3.438922e+01     8.571252e-01
 * time: 1.373725175857544
     4    -4.800331e+01     5.472675e-01
 * time: 1.844722032546997
     5    -5.708601e+01     1.845927e-01
 * time: 2.1926350593566895
     6    -5.990736e+01     1.078870e-01
 * time: 2.2947142124176025
     7    -6.094304e+01     5.303880e-02
 * time: 2.405125141143799
     8    -6.134989e+01     7.576809e-02
 * time: 2.5138230323791504
     9    -6.166452e+01     3.774805e-02
 * time: 2.620974063873291
    10    -6.187469e+01     2.881398e-02
 * time: 2.728518009185791
    11    -6.203117e+01     2.117476e-02
 * time: 2.830573081970215
    12    -6.209535e+01     2.121920e-02
 * time: 2.9375131130218506
    13    -6.215499e+01     1.391539e-02
 * time: 3.045436143875122

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557667
    AtomicNonlocal      14.8522633
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666460054
